In [1]:
with open("real_estate_sales_data.txt") as f:
  real_estate_sales = f.read()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
  separator = r'\d+\.',
  chunk_size = 100,
  chunk_overlap = 0,
  length_function = len,
  is_separator_regex = True
)

docs = text_splitter.create_documents([real_estate_sales])

docs[0]

Document(page_content='[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。', metadata={})

In [4]:
len(docs)

70

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

query =  "小区吵不吵"

answer_list = db.similarity_search(query)

for ans in answer_list:
  print(ans.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。

[客户问题] 我担心小区会很拥挤。
[销售回答] 这个小区总体规划非常合理，保证了每个单元之间有足够的空间。



In [8]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [9]:
db.save_local("real_estate_sale")

In [10]:
topK_retriever = db.as_retriever(search_kwargs={"k":3})
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x15f03fc50>, search_type='similarity', search_kwargs={'k': 3})

In [11]:
docs_topK = topK_retriever.get_relevant_documents(query)
for doc in docs_topK:
  print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。



In [12]:
docs_topK_house = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

for doc in docs_topK_house:
  print(doc.page_content + "\n")

[客户问题] 你们会提供家具吗？
[销售回答] 我们的精装房会提供基础家具和家电，让您拎包入住。

[客户问题] 都有哪些户型？
[销售回答] 我们有从一室到四室不等的多种户型，定能满足您不同的居住需求。

[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。



In [13]:
retriever = db.as_retriever(
  search_type = "similarity_score_threshold",
  search_kwargs = {"score_threshold": 0.8}
)

docs_similarity = retriever.get_relevant_documents(query)
for doc in docs_similarity:
  print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。



In [14]:
docs = retriever.get_relevant_documents(query)
docs[0].page_content

'[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [15]:
docs[0].page_content.split("[销售回答]")

['[客户问题] 我担心楼下太吵。\n', ' 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [16]:
ans = docs[0].page_content.split("[销售回答] ")[-1]
print(ans)

这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。


In [17]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
  retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})
  docs = retriever.get_relevant_documents(query)
  ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]
  return ans_list

query = "我想离医院近点"

print(sales(query))

[]


/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [18]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [19]:
query = "价格200万以内"

print(f"score: 0.8, ans: {sales(query)}\n")
print(f"score: 0.75, ans: {sales(query,0.75)}\n")
print(f"score: 0.5, ans: {sales(query, 0.5)}\n")

/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score: 0.8, ans: []



/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score: 0.75, ans: []

score: 0.5, ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



In [20]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
  llm, 
  retriever=db.as_retriever(search_type="similarity_score_threshold",
                            search_kwargs={"score_threshold": 0.8})
)

qa_chain({"query": query})


/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '价格200万以内', 'result': '请问您是在询问什么产品或服务的价格？'}

In [21]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以一般情况下，小区内部的噪音会比较低。'}

In [22]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


In [24]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estate_sale", OpenAIEmbeddings())

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
  llm, 
  retriever=db.as_retriever(search_type="similarity_score_threshold", 
                            search_kwargs={"score_threshold": 0.75}))

qa_chain({"query": "我想买别墅，你们有么"})

/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


{'query': '我想买别墅，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供房地产销售服务。建议您联系当地的房地产中介或者房产开发商，他们可以帮助您找到合适的别墅。'}

In [25]:
qa_chain.combine_documents_chain.verbose = True
qa_chain({"query": "我想买别墅，你们有么"})

/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供房地产销售服务。建议您联系当地的房地产中介或者房产开发商，他们可以帮助您找到合适的别墅。'}

In [26]:
qa_chain_return_source_documents = True
result = qa_chain({"query": "我想买别墅，你们有么"})

/Users/escray/miniconda3/envs/geektime/lib/python3.11/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [27]:
result

{'query': '我想买别墅，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供房地产买卖服务。建议您咨询房地产中介或者在房地产网站上搜索别墅的信息。'}

In [2]:
%pip install gradio

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/73/0a/792d54defeefbe900140bb56f08b8375f4bbe240ed534a42bb6364989b5d/gradio-3.44.3-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles<24.0,>=22.0 from https://files.pythonhosted.org/packages/c5/19/5af6804c4cc0fed83f47bff6e413a98a36618e7d40185cd36e69737f3b0e/aiofiles-23.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6.0,>=4.2.0 from https://files.pythonhosted.org/packages/f2/b4/02a0221bd1da91f6e6acdf0525528db24b4b326a670a9048da474dfe0667/altair-5.1.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for gradio-client==0.5.0 from https://files.pythonhosted.org/packages/fe/85/ec0323f39192c4bee04e8e06e64213aff816b9d1b61c3c8367e75b1c7e10/gradio_client-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx from https://files.pythonhosted.org/packages/33/0d/d9ce469af019741c8999711d

In [4]:
import gradio as gr
import random
import time
from typing import List

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
